In [1]:
from gates_multimodes import *
from states import *
from measurements_multimodes import *
from utils import *
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange
import cma

In [2]:
gate = D_gate(1., 1., 1, 2, 3)

In [3]:
gate = CX_gate(1.,1,2,2,3)

In [4]:
state1 = np.zeros(3)
state1[0] = 1
state2 = np.zeros(3)
state2[0] = 1

In [5]:
state = np.kron(state1, state2)
state = np.einsum('ij,j->i',gate,state)

In [6]:
state = state.reshape(state.shape[0],1)
rho = np.dot(state,state.conj().T)

In [7]:
rho.shape

(9, 9)

In [8]:
ptrace(rho, 0, 2)

array([[0.82547972, 0.        , 0.10078938],
       [0.        , 0.11055514, 0.        ],
       [0.10078938, 0.        , 0.06396514]])